<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#決定木学習" data-toc-modified-id="決定木学習-1">決定木学習</a></span><ul class="toc-item"><li><span><a href="#不純度" data-toc-modified-id="不純度-1.1">不純度</a></span><ul class="toc-item"><li><span><a href="#ジニ不順度-($I_G$)" data-toc-modified-id="ジニ不順度-($I_G$)-1.1.1">ジニ不順度 ($I_G$)</a></span></li><li><span><a href="#エントロピー-($I_H$)" data-toc-modified-id="エントロピー-($I_H$)-1.1.2">エントロピー ($I_H$)</a></span></li><li><span><a href="#分類誤差-($I_E$)" data-toc-modified-id="分類誤差-($I_E$)-1.1.3">分類誤差 ($I_E$)</a></span></li></ul></li></ul></li><li><span><a href="#アンサンブル学習" data-toc-modified-id="アンサンブル学習-2">アンサンブル学習</a></span><ul class="toc-item"><li><span><a href="#バギング" data-toc-modified-id="バギング-2.1">バギング</a></span></li><li><span><a href="#ブースティング" data-toc-modified-id="ブースティング-2.2">ブースティング</a></span></li><li><span><a href="#アダブースト-(Adaptive-Boost)" data-toc-modified-id="アダブースト-(Adaptive-Boost)-2.3">アダブースト (Adaptive Boost)</a></span></li></ul></li></ul></div>

# 決定木学習

情報利得と呼ばれる指標を用いて、分類問題を解いていく。二分木であれば「Yes/No」を何度も繰り返していくことは、馴染みがある。
情報利得をInformation Gain、データセットをD、$N$を各ノードのデータ点の総数であるとして、以下の式で情報利得を考える。

$$
IG(D_p, f) = I(D_p) - \sum_{j_1}^m \frac{N_j}{N_p}I(D_j)
$$

$I$は不純度（impurity）と呼ばれる指標である。例えば二分木であれば一回分割したあとクラスは2つできるので、$j=1,2$である。決定木学習ではこの情報利得が最大となる場所で分割を行って、次のノードの処理に移る。（言い換えれば不純度が最小となるように、うまく閾値を設定すると考えても良い）

一般的には分割したノードには正解と不正解が混ざっており、どれくらいの割合で混じっているか、を不順度で表す。よく使用されるものとして次の3種類がある：

## 不純度

どの不純度を使うのがいいか、ということに最適化の研究をするのは時間の無駄（らしい）。

### ジニ不順度 ($I_G$)

$$
I_G(t) = \sum_{i=1}^c p(i|t)(1-p(i|t))
$$


### エントロピー ($I_H$)

確率変数$x$のエントロピーは

$$
H(x) = - \sum_x p(x)\log_2p(x)
$$

で定義される。
特定のノード$t$の中で、クラス$i$に所属しているデータ点の割合を $p(i|t)$で記すと、エントロピーは次のように書き直せる。

$$
I_H(t) = - \sum_{i=1}^c p(i|t)\log_2p(i|t)
$$

二分木であれば $c=2$である。

### 分類誤差 ($I_E$)

$$
I_E(t) = 1 - \max {p(i|t)}
$$

# アンサンブル学習

単一の分類器を使用するのではなく、複数の分類器の結果を組み合わせることでより良い性能を達成するのがアンサンブル法の目的である。どのように組み合わせるか、によって（アンサンブルの手法によって）種類が異なってくる。分類器は何でも良いが（回帰分析でもok）、基本は決定木が使用される。

- 多数決（純粋に最も多い結果を全体の結果として解釈する）
- バギング
- アダブースト
- 勾配ブースティング（XGBoost, LightGBM)

## バギング

訓練データセットをランダムな復元抽出（ブートストラップ標本の作成）を行い、分類器を学習させる。この抽出のことをバギングと呼び、それぞれのサブセットで訓練した結果を、最終的には多数決で組み合わせる。

## ブースティング

バギングとは対象的に、ランダムに非復元抽出（重複なし）されたサンプルを用いて学習させる。この重複なし、というのは1回の抽出の際についてのことである。抽出間では重複については考慮しない。原始的なブースティング手法では、以上のように学習させた分類器を多数決で組み合わせる。

よく使用されるアダブーストでは抽出は行わず、全ての訓練データセットを使用して学習を行う。

## アダブースト (Adaptive Boost)

ブースティングの手法としてよく使用される。同じデータセットに対して何度も学習を行い、それらの学習結果を重みづけて多数決を取り組み合わせる手法。


1. 重みベクトルを $w_i =1/N$ で初期化する。$\sum_i w_i = 1$が成り立つ

2. 弱学習器を訓練する。

データ$X$、重み$w$、正解ラベル$y$を用いて弱分類器を作成する。

$$
C_j = \mathrm{train}(X, y, w)
$$

3. クラスラベルを予測する $\hat{y}$

4. 誤分類率の計算 

$$
\epsilon = \vec{w} \times (\hat{y} \neq y)
$$

5. 重みの更新に用いる係数を計算する 

$$
\alpha_j = 0.5 \mathrm{log} \frac{1-\epsilon}{\epsilon}
$$

6. 重みを更新する （$\hat{y}$は予測クラスラベル、$y$は真のクラスラベル）

$$
w = w \times \mathrm{exp}\left( -\alpha_j \times \hat{y} \times y \right)
$$

7. 重みを規格化する

$$
w = \frac{w}{\sum w_i}
$$

8. 入力された特徴量の行列に対する最終予測は以下のように計算する。各手順で推定した係数 $\alpha_j$ で予測結果を重み付ける

$$
sign\left(\sum_{j=1}^m \alpha_j \times \mathrm{predict}(C_j, X) \right)
$$

この結果が０よりも大きければクラス1、小さければクラス-1とする。関数$sign$は符号関数と呼ばれ、引数が正なら1、0なら0、負なら-1を返す関数である。